In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-26 19:39:14--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-26 19:39:14 (9.80 MB/s) - ‘minsearch.py’ saved [3832/3832]



#### Minisearch

In [27]:
import minsearch
import json
with open('simplified_national_parks_data.json', 'rt') as f_in:
    docs_raw = json.load(f_in)


#### Elastic Search

In [28]:
import elasticsearch as es
index_name = 'parks'

https://github.com/DataTalksClub/llm-zoomcamp/blob/main/02-open-source/huggingface-mistral-7b.ipynb

In [29]:
from elasticsearch import Elasticsearch, helpers

# Initialize Elasticsearch client
# es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
# Connect without SSL
es = Elasticsearch("http://localhost:9200")



In [30]:
es

<Elasticsearch(['http://localhost:9200'])>

In [31]:

# Define your index schema (mapping) with additional fields
index_name = "parks"
mapping = {
    "mappings": {
        "properties": {
            "description": {"type": "text"},
            "activities": {"type": "text"},
            "topics": {"type": "text"},
            "weatherInfo": {"type": "text"},
            "fullName": {"type": "text"},
            "parkCode": {"type": "keyword"},
            "states": {"type": "text"},
            "contacts": {"type": "text"},  # Simplified contacts as text
            "url": {"type": "text"},
            "directionsInfo": {"type": "text"},
            "directionsUrl": {"type": "text"},
            "addresses": {"type": "text"},
            # "images": {
            #     "type": "nested",  # Since images are a list of objects
            #     "properties": {
            #         "title": {"type": "text"},
            #         "url": {"type": "text"}
            #     }
            # }
        }
    }
}

# Check if the index exists and create it if it doesn't
if es.indices.exists(index=index_name):
    print(f"Index '{index_name}' already exists. Deleting and recreating it.")
    es.indices.delete(index=index_name)

# Create the index with the mapping
es.indices.create(index=index_name, body=mapping)


# Prepare the documents (in the format Elasticsearch expects)
docs = [
    {
        '_index': index_name,
        '_source': {
            'id': doc['id'],
            'url': doc['url'],
            'fullName': doc['fullName'],
            'parkCode': doc['parkCode'],
            'description': doc['description'],
            'activities': ' '.join(doc['activities']),  # Join activities into a single string
            'topics': ' '.join(doc['topics']),  # Join topics into a single string
            'states': doc['states'],
            'contacts': doc['contacts'],  # Using the simplified contacts as text
            'directionsInfo': doc['directionsInfo'],
            'directionsUrl': doc['directionsUrl'],
            'addresses': doc['addresses'],  # Simplified addresses as text
            # 'images': doc['images'],  # Nested images structure
            'weatherInfo': doc.get('weatherInfo', '')
        }
    } for doc in docs_raw
]

# Bulk insert the documents into Elasticsearch
helpers.bulk(es, docs)

print("Data indexed successfully.")


Index 'parks' already exists. Deleting and recreating it.
Data indexed successfully.


In [32]:
def search_elasticsearch(query, filters=None, boost_fields=None, num_results=10):
    """
    Search Elasticsearch index with optional filters and boosting.

    Args:
        query (str): The search query string.
        filters (dict): Keyword fields to filter by.
        boost_fields (dict): Fields to apply boosting.
        num_results (int): Number of results to return.

    Returns:
        list of dict: List of matched documents with scores.
    """
    search_body = {
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["description", "activities", "topics", "weatherInfo", "fullName", "addresses", "url"]
                    }
                },
                "filter": [
                    {"term": {key: value}} for key, value in (filters or {}).items()
                ]
            }
        },
        "size": num_results
    }

    # Add boosting if specified
    if boost_fields:
        must_queries = [
            {
                "multi_match": {
                    "query": query,
                    "fields": ["description", "activities", "topics", "weatherInfo", "fullName", "addresses", "url"]
                }
            }
        ]
        
        for field, boost in boost_fields.items():
            must_queries.append({
                "match": {
                    field: {
                        "query": query,
                        "boost": boost
                    }
                }
            })

        search_body['query']['bool']['must'] = must_queries

    # Perform the search
    try:
        response = es.search(index=index_name, body=search_body)
        return [{"_source": hit['_source'], "_score": hit['_score']} for hit in response['hits']['hits']]
    except Exception as e:
        print(f"Error occurred while searching: {e}")
        return []

In [33]:

# Example search query
results = search_elasticsearch(query="smoky mountains", boost_fields={"description": 2.0}, num_results=5)
print(results)

[{'_source': {'id': 'D9819727-18DF-4A84-BDDE-D4F2696DE340', 'url': 'https://www.nps.gov/grsm/index.htm', 'fullName': 'Great Smoky Mountains National Park', 'parkCode': 'grsm', 'description': "Ridge upon ridge of forest straddles the border between North Carolina and Tennessee in Great Smoky Mountains National Park. World renowned for its diversity of plant and animal life, the beauty of its ancient mountains, and the quality of its remnants of Southern Appalachian mountain culture, this is America's most visited national park. Plan your visit today!", 'activities': 'Arts and Culture Live Music Auto and ATV Scenic Driving Astronomy Stargazing Biking Road Biking Camping Backcountry Camping Car or Front Country Camping Horse Camping (see also Horse/Stock Use) Group Camping RV Camping Fishing Freshwater Fishing Fly Fishing Food Picnicking Guided Tours Self-Guided Tours - Auto Hands-On Citizen Science Hiking Backcountry Hiking Front-Country Hiking Horse Trekking Horse Camping (see also camp

In [34]:
count_response = es.count(index=index_name)
print("Total documents in index:", count_response['count'])


Total documents in index: 62


In [35]:
# from getpass import getpass

# HUGGINGFACEHUB_API_TOKEN = getpass()

Read HF token from env varaibles

In [36]:
from dotenv import load_dotenv
from pathlib import Path
import os
# load_dotenv(dotenv_path=Path(__file__).resolve().parent.parent / '.env')
load_dotenv(dotenv_path='.env')

True

In [37]:
HUGGINGFACEHUB_API_TOKEN = os.getenv('HF_TOKEN')

In [38]:
from langchain_huggingface import HuggingFaceEndpoint

llm_h = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\h4has\.cache\huggingface\token
Login successful


In [39]:
from langchain_huggingface import HuggingFaceEndpoint
from elasticsearch import Elasticsearch

# Initialize Hugging Face client
llm_h = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

# Initialize Elasticsearch client
# es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# Search function to query Elasticsearch
def search_elasticsearch(query, filters=None, num_results=2):
    search_body = {
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["description", "activities", "topics", "weatherInfo", "fullName", "states", "addresses", "url"]
                    }
                },
                "filter": [
                    {"term": {key: value}} for key, value in (filters or {}).items()
                ]
            }
        },
        "size": num_results
    }

    response = es.search(index="parks", **search_body)  # Updated to use parameter unpacking
    return [hit['_source'] for hit in response['hits']['hits']]

# Function to build the prompt for Hugging Face
def build_prompt(query, search_results):
    prompt_template = """
You're a park information assistant. Answer the QUESTION based on the CONTEXT from the parks database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"Full Name: {doc['fullName']}\nDescription: {doc['description']}\nActivities: {doc['activities']}\nTopics: {doc['topics']}\nWeather Info: {doc['weatherInfo']}\nAddress: {doc['addresses']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

# Function to call Hugging Face API
def llm(prompt):
    print('\n\n[DEBUG] Calling HF API for llm response...')
    response = llm_h.invoke(prompt) 
    print(response)
    return response  

# Main RAG function
def rag(query):
    print('\n\n[DEBUG] Running RAG Pipeline...')
    # Perform search to get relevant documents
    search_results = search_elasticsearch(query)
    # Build prompt with search results
    prompt = build_prompt(query, search_results)
    print(prompt)
    # Get answer from Hugging Face model
    answer = llm(prompt)
    return answer

# Example query
query = 'What activities can I do at Acadia National Park?'
answer = rag(query)
print(answer)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\h4has\.cache\huggingface\token
Login successful


[DEBUG] Running RAG Pipeline...
You're a park information assistant. Answer the QUESTION based on the CONTEXT from the parks database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: What activities can I do at Acadia National Park?

CONTEXT: 
Full Name: Great Smoky Mountains National Park
Description: Ridge upon ridge of forest straddles the border between North Carolina and Tennessee in Great Smoky Mountains National Park. World renowned for its diversity of plant and animal life, the beauty of its ancient mountains, and the quality of its remnants of Southern Appalachian mountain culture, this is America's most 

In [40]:
query = 'how many national parks are there in Kentucky state ? give me only full names of them. '
answer = rag(query)
print(answer)



[DEBUG] Running RAG Pipeline...
You're a park information assistant. Answer the QUESTION based on the CONTEXT from the parks database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: how many national parks are there in Kentucky state ? give me only full names of them. 

CONTEXT: 
Full Name: Redwood National and State Parks
Description: Most people know Redwood as home to the tallest trees on Earth. But the Parks also protect vast prairies, oak woodlands, wild rivers, and 40 miles of rugged coastline. People have lived in this verdant landscape since time immemorial. Together, the National Park Service and California State Parks are managing and restoring these lands for the inspiration, enjoyment, and education of all.
Activities: Arts and Culture Cultural Demonstrations Auto and ATV Scenic Driving Biking Mountain Biking Road Biking Camping Backcountry Camping Car or Front Country Camping Horse Camping (see also Horse/Stock Use) RV Camping Fishing Saltwat

In [41]:
query = 'In which state channel islands is located ?'
answer = rag(query)
print(answer)



[DEBUG] Running RAG Pipeline...
You're a park information assistant. Answer the QUESTION based on the CONTEXT from the parks database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: In which state channel islands is located ?

CONTEXT: 
Full Name: Channel Islands National Park
Description: Channel Islands National Park encompasses five remarkable islands and their ocean environment, preserving and protecting a wealth of natural and cultural resources. Isolation over thousands of years has created unique animals, plants, and archeological resources found nowhere else on Earth and helped preserve a place where visitors can experience coastal southern California as it once was.
Activities: Astronomy Stargazing Boating Motorized Boating Sailing Boat Tour Camping Backcountry Camping Canoe or Kayak Camping Group Camping Compass and GPS Orienteering Fishing Saltwater Fishing Food Picnicking Guided Tours Self-Guided Tours - Walking Boat Tour Hiking Backcountry Hi

In [74]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [75]:
len(model.encode("This is a simple sentence"))


768